In [5]:
# 导入必要的库
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

# 忽略所有警告
warnings.filterwarnings('ignore')

In [6]:
# 设置文件路径
amihud_path = './amihud.pkl'
ede_excel_path = './EDE20241125.xlsx'  # 主板股票文件
price_month_path = './price_month.pkl'
dfin_path = './dfin_final.csv'
fs_combas_path = './FS_Combas.xlsx'  # 总资产数据文件

# 输出目录
output_dir = Path('./output')
output_dir.mkdir(exist_ok=True)

In [9]:
%%time
# ------------------- 读取数据 -------------------

# 1. 读取 amihud.pkl
amihud_df = pd.read_pickle(amihud_path)
print(f"amihud数据行数: {amihud_df.shape[0]}")
print(f"amihud数据时间范围: {amihud_df['year'].min()}-{amihud_df['month'].min()} 到 {amihud_df['year'].max()}-{amihud_df['month'].max()}")

# 2. 读取 EDE20241125.xlsx，用于标记主板股票
ede_df = pd.read_excel(ede_excel_path)

# 提取主板股票代码，确保为6位字符串
if '股票代码' not in ede_df.columns:
    raise KeyError("EDE20241125.xlsx中缺少 '股票代码' 列。请检查文件格式。")

# 转换股票代码为字符串格式，并补全6位
ede_df['股票代码'] = ede_df['股票代码'].astype(str).str.split('.').str[0].str.zfill(6)
mainboard_codes = ede_df['股票代码'].unique()
print(f"主板股票代码数量: {len(mainboard_codes)}")
print(f"主板股票代码样例: {mainboard_codes[:5]}")

# 3. 读取 price_month.csv，选择需要的列，假设第一列是无用的索引列
# 需要的列为 'code', 'year', 'month', 'close_price', 'market_value', 'return', 'total_value', 'risk_free_month'
price_month_df = pd.read_pickle(
    price_month_path
)
price_month_df = price_month_df[['code', 'year', 'month', 'close_price', 'market_value', 'return', 'total_value', 'risk_free_month']]
price_month_df['code'] = price_month_df['code'].astype(str).str.zfill(6)  # 补全前导零至6位
price_month_df['risk_free_month'] = price_month_df['risk_free_month']*0.01
initial_price_month_rows = price_month_df.shape[0]
price_month_df.drop_duplicates(subset=['code', 'year', 'month'], inplace=True)
print(f"price_month去重前行数: {initial_price_month_rows}, 去重后行数: {price_month_df.shape[0]}")

# 4. 读取 dfin_final.csv，选择需要的列，假设第一列是无用的索引列
# 需要的列为 'code', 'year', 'month', 'dfin'
dfin_df = pd.read_csv(
    dfin_path,
    usecols=['code', 'year', 'month', 'dfin'],
    dtype={'code': str}
)
dfin_df['code'] = dfin_df['code'].str.zfill(6)  # 补全前导零至6位
initial_dfin_rows = dfin_df.shape[0]
dfin_df.drop_duplicates(subset=['code', 'year', 'month'], inplace=True)
print(f"dfin数据去重前行数: {initial_dfin_rows}, 去重后行数: {dfin_df.shape[0]}")

# ------------------- 读取并处理 FS_Combas.xlsx -------------------

# 5. 读取总资产数据
fs_df = pd.read_excel(fs_combas_path)

# 筛选 Typrep == 'A' 的合并报表
fs_df = fs_df[fs_df['Typrep'] == 'A']
print(f"筛选合并报表后的总资产数据行数: {fs_df.shape[0]}")

# 转换股票代码为字符串格式，并补全6位
fs_df['code'] = fs_df['Stkcd'].astype(str).str.zfill(6)

# 转换 Accper 为日期格式，并提取 year 和 month
fs_df['Accper'] = pd.to_datetime(fs_df['Accper'])
fs_df['year'] = fs_df['Accper'].dt.year
fs_df['month'] = fs_df['Accper'].dt.month
fs_df['day'] = fs_df['Accper'].dt.day

# 删除连续日期为上月最后一天和下月第一天的重复数据
fs_df = fs_df.sort_values(by=['code', 'Accper'])
fs_df['prev_Accper'] = fs_df.groupby('code')['Accper'].shift(1)
fs_df['date_diff'] = (fs_df['Accper'] - fs_df['prev_Accper']).dt.days
# 假设一个月最多有31天，且月末前一日到月初一日的差为1天
fs_df = fs_df[~((fs_df['date_diff'] == 1) & (fs_df['day'] == 1))]

# 删除辅助列
fs_df.drop(columns=['prev_Accper', 'date_diff', 'day'], inplace=True)

# 转换 A001000000 为亿元
fs_df['A001000000'] = fs_df['A001000000'] / 1e8  # 转换为亿元
fs_df.rename(columns={'A001000000': 'total_assets'}, inplace=True)

print(f"处理后的总资产数据行数: {fs_df.shape[0]}")

# ------------------- 将总资产数据与 dfin 进行匹配并标准化 -------------------

# 6. 为 dfin_df 添加日期列
dfin_df['date'] = pd.to_datetime(dfin_df['year'].astype(str) + '-' + dfin_df['month'].astype(str) + '-01')

# 7. 将 fs_df 的数据日期向后移动一个月，表示使用上一期的总资产
fs_df['next_date'] = fs_df.groupby('code')['Accper'].shift(-1)
fs_df['next_total_assets'] = fs_df.groupby('code')['total_assets'].shift(-1)

# 8. 创建总资产展开的数据
total_assets_expanded = []

for code, group in fs_df.groupby('code'):
    group = group.sort_values(by='Accper')
    for idx, row in group.iterrows():
        start_date = row['Accper'] + pd.Timedelta(days=1)
        if pd.notna(row['next_date']):
            end_date = row['next_date'] - pd.Timedelta(days=1)
        else:
            end_date = pd.Timestamp('2024-12-31')  # 设定一个结束日期
        
        ta = row['next_total_assets']
        if pd.isna(ta):
            ta = row['total_assets']  # 如果没有下一期总资产，使用当前期总资产
        
        # 生成每月的开始日期
        date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
        temp_df = pd.DataFrame({
            'code': code,
            'date': date_range,
            'prev_total_assets': ta
        })
        total_assets_expanded.append(temp_df)

# 合并所有展开的数据
if total_assets_expanded:
    total_assets_df = pd.concat(total_assets_expanded, ignore_index=True)
    print(f"总资产展开后的数据行数: {total_assets_df.shape[0]}")
else:
    total_assets_df = pd.DataFrame(columns=['code', 'date', 'prev_total_assets'])
    print("总资产展开后的数据行数: 0")

# 9. 将 total_assets_df 与 dfin_df 合并
dfin_df = pd.merge(dfin_df, total_assets_df, on=['code', 'date'], how='left')
print(f"dfin与总资产合并后的数据行数: {dfin_df.shape[0]}")
print(f"dfin与总资产合并后的缺失值数: {dfin_df['prev_total_assets'].isna().sum()}")

# 10. 删除缺失 prev_total_assets 的行
dfin_df = dfin_df.dropna(subset=['prev_total_assets'])
print(f"删除缺失prev_total_assets后的数据行数: {dfin_df.shape[0]}")

# 11. 计算标准化的 dfin
dfin_df['dfin_standardized'] = dfin_df['dfin'] / dfin_df['prev_total_assets']
print(f"标准化后的dfin的描述统计:\n{dfin_df['dfin_standardized'].describe()}")

# 12. 删除辅助列
dfin_df.drop(columns=['prev_total_assets', 'date'], inplace=True)

# ------------------- 继续数据处理 -------------------

# 13. 确保 dfin_standardized 数据按股票代码和时间排序
dfin_df = dfin_df.sort_values(by=['code', 'year', 'month'])

# 14. 创建完整的年月范围
full_years = range(dfin_df['year'].min(), dfin_df['year'].max() + 1)
full_months = range(1, 13)
full_date_index = pd.MultiIndex.from_product(
    [dfin_df['code'].unique(), full_years, full_months],
    names=['code', 'year', 'month']
)

# 15. 使用 MultiIndex 重新索引，确保每只股票在所有年份和月份都有记录
dfin_df = dfin_df.set_index(['code', 'year', 'month']).reindex(full_date_index).reset_index()

# 16. 再次进行前向填充，确保所有空缺月份使用最新的 dfin_standardized 值
dfin_df['dfin_standardized'] = dfin_df.groupby('code')['dfin_standardized'].ffill()

# 17. 检查填充后的缺失值情况
print(f"dfin重新索引并填充后的缺失值数: {dfin_df['dfin_standardized'].isna().sum()}")

# ------------------- 合并数据 -------------------

# 18. 标记主板股票
mainboard_df = pd.DataFrame({'code': mainboard_codes, 'is_main_board': True})
merged_df = pd.merge(amihud_df, mainboard_df, on='code', how='left')
merged_df['is_main_board'] = merged_df['is_main_board'].fillna(False)
print(f"主板股票标记数量: {merged_df['is_main_board'].sum()}")

# 19. 处理日期格式，确保 year 和 month 为整数类型
merged_df['year'] = merged_df['year'].astype(int)
merged_df['month'] = merged_df['month'].astype(int)

# 20. 合并价格数据，基于 code、year 和 month
merged_df = pd.merge(merged_df, price_month_df, on=['code', 'year', 'month'], how='left')
print(f"合并价格数据后的数据行数: {merged_df.shape[0]}")
print(f"价格数据缺失值数: {merged_df['close_price'].isna().sum()}")

# 21. 合并标准化后的 dfin 数据，基于 code、year 和 month
merged_df = pd.merge(merged_df, dfin_df, on=['code', 'year', 'month'], how='left')
print(f"合并标准化后的dfin数据后的数据行数: {merged_df.shape[0]}")
print(f"标准化后的dfin缺失值数: {merged_df['dfin_standardized'].isna().sum()}")

# 22. 删除缺失 dfin_standardized 的行
merged_df = merged_df.dropna(subset=['dfin_standardized'])
print(f"删除缺失dfin_standardized后的数据行数: {merged_df.shape[0]}")

# 23. 删除缺失 return 或 market_value 的行
initial_rows = merged_df.shape[0]
merged_df = merged_df.dropna(subset=['return', 'market_value'])
print(f"删除缺失return或market_value前行数: {initial_rows}, 删除后行数: {merged_df.shape[0]}")

# 24. 添加 year_month 列，格式为 "YYYY-MM"
merged_df['year_month'] = merged_df['year'].astype(str) + '-' + merged_df['month'].astype(str).str.zfill(2)
print("year_month列已添加")



amihud数据行数: 636494
amihud数据时间范围: 2005-1 到 2024-12
主板股票代码数量: 3173
主板股票代码样例: ['000001' '000002' '000004' '000006' '000007']
price_month去重前行数: 818205, 去重后行数: 756603
dfin数据去重前行数: 58036, 去重后行数: 58036
筛选合并报表后的总资产数据行数: 338696
处理后的总资产数据行数: 277531
总资产展开后的数据行数: 925189
dfin与总资产合并后的数据行数: 58036
dfin与总资产合并后的缺失值数: 0
删除缺失prev_total_assets后的数据行数: 58036
标准化后的dfin的描述统计:
count    5.803600e+04
mean             -inf
std               NaN
min              -inf
25%     -5.591565e+06
50%     -4.886176e+05
75%      3.170984e+06
max      4.257925e+11
Name: dfin_standardized, dtype: float64
dfin重新索引并填充后的缺失值数: 1367919
主板股票标记数量: 473854
合并价格数据后的数据行数: 636494
价格数据缺失值数: 47920
合并标准化后的dfin数据后的数据行数: 636494
标准化后的dfin缺失值数: 70708
删除缺失dfin_standardized后的数据行数: 565786
删除缺失return或market_value前行数: 565786, 删除后行数: 520904
year_month列已添加
CPU times: total: 2min 32s
Wall time: 7min 21s


In [10]:
# ------------------- 构造投资组合 -------------------

def calculate_investment_portfolio(data, value_column, period_name):
    """
    根据指定的指标计算投资组合
    :param data: 合并后的数据
    :param value_column: 用于计算投资组合的指标列名（如 'ami3', 'dfin_standardized'）
    :param period_name: 投资组合的名称，用于输出文件名
    """
    print(f"\n开始构造 {period_name} 投资组合")
    grouped = data.groupby(['year', 'month'])
    results = []

    for (year, month), group in grouped:
        year_month = f"{year}-{str(month).zfill(2)}"
        main_board_group = group[group['is_main_board']]
        main_board_count = main_board_group.shape[0]
        unique_values = main_board_group[value_column].nunique()
        print(f"月份 {year_month} - 主板股票数: {main_board_count}, {value_column}唯一值数: {unique_values}")
        
        # 确保主板股票数量和唯一值数目足够
        if unique_values < 5:
            print(f"跳过月份 {year_month}，因为{value_column}的主板股票唯一值数目少于5")
            continue
        
        # 计算分位数并分组
        try:
            # 使用主板股票计算分位数
            main_board_group = main_board_group.copy()
            main_board_group['quintile'] = pd.qcut(main_board_group[value_column], 5, labels=False, duplicates='drop')
        except Exception as e:
            print(f"跳过月份 {year_month}，分位数计算失败: {e}")
            continue
        
        # 创建 code 到 quintile 的映射
        quintile_map = main_board_group[['code', 'quintile']].drop_duplicates()
        
        # 合并 quintile_map 到所有股票
        group = pd.merge(group, quintile_map, on='code', how='left')
        
        # 做多最低分位组，做空最高分位组
        long_group = group[group['quintile'] == 0]
        short_group = group[group['quintile'] == 4]
        
        # 确保做多和做空组不为空
        if long_group.empty or short_group.empty:
            print(f"跳过月份 {year_month}，做多或做空组为空")
            continue
        
        # 计算流动市值加权收益
        try:
            # 流动市值加权
            long_weight = long_group['market_value'] / long_group['market_value'].sum()
            short_weight = short_group['market_value'] / short_group['market_value'].sum()
            
            return_long_mv = (long_weight * long_group['return']).sum()
            return_short_mv = (short_weight * short_group['return']).sum()
            anomaly_return_mv = return_long_mv - return_short_mv
            
            # 等权重收益
            return_long_eq = long_group['return'].mean()
            return_short_eq = short_group['return'].mean()
            anomaly_return_eq = return_long_eq - return_short_eq
            
            # 获取当前月的无风险收益率
            risk_free_rate = group['risk_free_month'].iloc[0] if 'risk_free_month' in group.columns else 0.0
            print(f"月份 {year_month} 的无风险收益率: {risk_free_rate}")
            
            # 计算做多组减做空组减无风险收益率
            anomaly_return_mv_rf = anomaly_return_mv - risk_free_rate
            anomaly_return_eq_rf = anomaly_return_eq - risk_free_rate
            
            # 存储结果
            results.append({
                'year_month': year_month,
                'return_long_mv': return_long_mv,
                'return_short_mv': return_short_mv,
                'risk_free_month': risk_free_rate,
                'anomaly_return_mv_rf': anomaly_return_mv_rf,
                'return_long_eq': return_long_eq,
                'return_short_eq': return_short_eq,
                'anomaly_return_eq_rf': anomaly_return_eq_rf
            })
        except Exception as e:
            print(f"计算收益时出错，月份 {year_month}: {e}")
            continue
    
    # 转为 DataFrame 并导出
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df = results_df.sort_values(by='year_month')
        # 选择输出列的顺序
        results_df = results_df[[
            'year_month',
            'return_long_mv',
            'return_short_mv',
            'risk_free_month',
            'anomaly_return_mv_rf',
            'return_long_eq',
            'return_short_eq',
            'anomaly_return_eq_rf'
        ]]
        results_df.to_excel(output_dir / f'{period_name}.xlsx', index=False)
        print(f"{period_name} 投资组合完成，生成记录数: {results_df.shape[0]}")
        print(f"{period_name} 投资组合已导出至 {output_dir / f'{period_name}.xlsx'}")
    else:
        print(f"{period_name} 投资组合结果为空，未生成任何记录。")

# ------------------- 计算投资组合 -------------------

# 25. 计算 ami3、ami6、ami9、ami12 投资组合
ami_periods = [3, 6, 9, 12]
for N in ami_periods:
    column_name = f'ami{N}'
    # 计算滚动平均 amihud，窗口为 N 个月，最小周期为 N，向前滚动，排除当前月
    merged_df[column_name] = merged_df.groupby('code')['amihud']\
        .rolling(window=N, min_periods=N).mean().shift(1).reset_index(level=0, drop=True)
    
    # 打印非缺失值数
    non_na_count = merged_df[column_name].notna().sum()
    print(f"{column_name} 计算完成，非缺失值数: {non_na_count}")
    
    # 计算投资组合
    calculate_investment_portfolio(merged_df, column_name, column_name)

# 26. 计算 dfin_standardized 投资组合
calculate_investment_portfolio(merged_df, 'dfin_standardized', 'dfin_standardized')

print("\n所有投资组合已完成，结果存储在 output 目录。")


ami3 计算完成，非缺失值数: 511119

开始构造 ami3 投资组合
月份 2005-06 - 主板股票数: 1075, ami3唯一值数: 1074
月份 2005-06 的无风险收益率: 0.001856
月份 2005-07 - 主板股票数: 1074, ami3唯一值数: 0
跳过月份 2005-07，因为ami3的主板股票唯一值数目少于5
月份 2005-08 - 主板股票数: 1073, ami3唯一值数: 0
跳过月份 2005-08，因为ami3的主板股票唯一值数目少于5
月份 2005-09 - 主板股票数: 1072, ami3唯一值数: 1072
月份 2005-09 的无风险收益率: 0.001856
月份 2005-10 - 主板股票数: 1070, ami3唯一值数: 1070
月份 2005-10 的无风险收益率: 0.001856
月份 2005-11 - 主板股票数: 1069, ami3唯一值数: 1068
月份 2005-11 的无风险收益率: 0.001856
月份 2005-12 - 主板股票数: 1063, ami3唯一值数: 1060
月份 2005-12 的无风险收益率: 0.001856
月份 2006-01 - 主板股票数: 1045, ami3唯一值数: 1040
月份 2006-01 的无风险收益率: 0.001856
月份 2006-02 - 主板股票数: 1028, ami3唯一值数: 1024
月份 2006-02 的无风险收益率: 0.001856
月份 2006-03 - 主板股票数: 1014, ami3唯一值数: 1014
月份 2006-03 的无风险收益率: 0.001856
月份 2006-04 - 主板股票数: 1073, ami3唯一值数: 1071
月份 2006-04 的无风险收益率: 0.001856
月份 2006-05 - 主板股票数: 1040, ami3唯一值数: 947
月份 2006-05 的无风险收益率: 0.001856
月份 2006-06 - 主板股票数: 1042, ami3唯一值数: 950
月份 2006-06 的无风险收益率: 0.001856
月份 2006-07 - 主板股票数: 1046, ami3唯一值数: 1043
月份 2006-0